#### Working with PlayWright Browser Toolkit

In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    base_url="http://localhost:11434",
    model = "qwen3:8b",
    temperature=0.5
)

/Users/roman_lagunov/agents-for-qa/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser,
)
import nest_asyncio
nest_asyncio.apply()


##### Get all Tools from Playwright Browser Toolkit

In [3]:
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit(async_browser=async_browser)
tools = toolkit.get_tools()
tools
tool_by_name = {tool.name: tool for tool in tools}
tool_by_name

{'click_element': ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/roman_lagunov/Library/Caches/ms-playwright/chromium-1187/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=140.0.7339.16>),
 'navigate_browser': NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/roman_lagunov/Library/Caches/ms-playwright/chromium-1187/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=140.0.7339.16>),
 'previous_webpage': NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/roman_lagunov/Library/Caches/ms-playwright/chromium-1187/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=140.0.7339.16>),
 'extract_text': ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/roman_lagunov/Library/Caches/ms-playwright/chromium-1187/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=140.0.7339.16>),
 'extract_hyperlinks': ExtractH

In [4]:
navigate_tool = tool_by_name["navigate_browser"]
get_elements_tool = tool_by_name["get_elements"]
navigate_tool

NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/roman_lagunov/Library/Caches/ms-playwright/chromium-1187/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=140.0.7339.16>)

In [5]:
await navigate_tool.arun("http://eaapp.somee.com/Employee")

'Navigating to http://eaapp.somee.com/Employee returned status code 200'

In [6]:
await get_elements_tool.arun({"selector": "td", "attributes": ["innerText"]})

'[{"innerText": "Karthik"}, {"innerText": "4000"}, {"innerText": "24"}, {"innerText": "1"}, {"innerText": "karthik@executeautomation.com"}, {"innerText": "Benefits Edit Delete"}, {"innerText": "Prashanth"}, {"innerText": "7000"}, {"innerText": "30"}, {"innerText": "2"}, {"innerText": "prashanth@executeautomation.com"}, {"innerText": "Benefits Edit Delete"}, {"innerText": "Ramesh"}, {"innerText": "3500"}, {"innerText": "13"}, {"innerText": "2"}, {"innerText": "ramesh@executeautomation.com"}, {"innerText": "Benefits Edit Delete"}, {"innerText": "John"}, {"innerText": "2500"}, {"innerText": "18"}, {"innerText": "3"}, {"innerText": "john@executeautomation.com"}, {"innerText": "Benefits Edit Delete"}, {"innerText": "Sarah Johnson"}, {"innerText": "5500"}, {"innerText": "36"}, {"innerText": "2"}, {"innerText": "sarah.johnson@executeautomation.com"}, {"innerText": "Benefits Edit Delete"}]'

#### Working with AI Agents

In [8]:
from langchain.agents import initialize_agent, AgentType

tools = toolkit.get_tools()

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = "Extract table data from http://eaapp.somee.com/Employee page and get me average salary of employees."
result = await agent.arun(query)
print(result)



> Entering new AgentExecutor chain...
<think>
Okay, let's see. The user wants to extract table data from the Employee page at http://eaapp.somee.com/Employee and find the average salary. First, I need to navigate to that URL. So the first action should be navigate_browser with the given URL.

Once there, I should check if there's a table on the page. Maybe I can extract all the text to see the structure. Alternatively, use get_elements with a selector like 'table' or 'tr' to get the table rows. Wait, tables are usually structured with <table> tags, and data in <tr> rows and <td> cells. So perhaps using get_elements with selector 'tr' to get each row.

But first, I need to confirm the page content. Let me start by navigating to the URL. Then, maybe extract all hyperlinks to see if there's a direct link to the table, but that might not be necessary. Alternatively, extract_text to see the content. However, if the table is in HTML, extracting text might not be the best approach. Maybe us